## Cyclistic Bike Share Company

### About the Company

In 2016, Cyclistic launched a successful bike-share offering. Since then, the program has grown
to a fleet of 5,824 bicycles that are geotracked and locked into a network of 692 stations
across Chicago. The bikes can be unlocked from one station and returned to any other station
in the system anytime.

### The Business Task

How do annual members and casual riders use Cyclistic bikes differently?

Note: Cyclistic’s finance analysts have concluded that annual members are much more profitable
than casual riders. Although the pricing flexibility helps Cyclistic attract more customers,
Moreno (Director of Marketing) believes that maximizing the number of annual members will be key to future growth. 

### Stakeholders and Team

Lily Moreno: The director of marketing 

Cyclistic marketing analytics team: A team of data analysts who are responsible for
collecting, analyzing, and reporting data that helps guide Cyclistic marketing strategy

Cyclistic executive team: The notoriously detail-oriented executive team will decide
whether to approve the recommended marketing program.

### Description of data sources
(12) 2023 monthly csv files containing 
ride_id: ID attached to each ride taken
rideable_type: bike type electric or classic
starttime: day and time trip started, in CST
stoptime: day and time trip ended, in CST
start_station_name
start_station_id
end_station_name
end_station_id
start_lat: starting latitude
start_lng:starting longitude
end_lat: ending latitude
end_lng: ending longitude
member_casual: type of rider membership or daypass

One csv file contining station information 
id: station id
name: station name
latitude of the station 
longitude of the station
dpc capacity: how many docking stations 
landmark
online data: the date the station went live in the system

### This is a ficticious Case Study 
(Note: The datasets have a different name because Cyclistic
is a fictional company.

### DATA 
 The data has been made available by
Motivate International Inc. under this license.https://divvybikes.com/data-license-agreement

### Choice for this notebook
I am going to try this in SQL lite to showcase my skills and my thought processes. 

In [ ]:
import pandas as pd
import sqlite3
import os

In [ ]:
# Create a connection to a new SQLite database
conn = sqlite3.connect('cyclistic_bike_share.db')

In [ ]:
# List of CSV files
csv_files = [
    '202301-divvy-tripdata.csv',
    '202302-divvy-tripdata.csv',
    '202303-divvy-tripdata.csv',
    '202304-divvy-tripdata.csv',
    '202305-divvy-tripdata.csv',
    '202306-divvy-tripdata.csv',
    '202307-divvy-tripdata.csv',
    '202308-divvy-tripdata.csv',
    '202309-divvy-tripdata.csv',
    '202310-divvy-tripdata.csv',
    '202311-divvy-tripdata.csv',
    '202312-divvy-tripdata.csv',
    'Divvy_Stations_2013.csv'
]

# Initializing and empty dataframe
bike_data = pd.DataFrame()

# Loading each CSV file and combine into a single DataFrame for analysis
for file in csv_files[:-1]:  # Exclude the stations file for the combined trip data
    df = pd.read_csv(f'/kaggle/input/cyclistic-bike-share-case-study/{file}')
    bike_data = pd.concat([bike_data, df])

## Creating a new table 

In [ ]:
bike_data.to_sql('combined_tripdata', conn, index=False, if_exists='replace')

I want to verify that the data got combined


In [ ]:
query = "SELECT COUNT(*) AS total_trips FROM combined_tripdata"
result = pd.read_sql(query, conn)
print(result)

### Analysis

First I want to see what the average trip length is for a member vs. a casual rider

In [ ]:
query = """
SELECT 
    member_casual,
    AVG((JULIANDAY(ended_at) - JULIANDAY(started_at)) * 24 * 60) AS average_trip_length_minutes
FROM 
    combined_tripdata
GROUP BY 
    member_casual;
"""

In [ ]:
result = pd.read_sql(query, conn)
print(result)

Next I want to see what type of bike each type of member uses most

In [ ]:
query = """
SELECT 
    member_casual,
    rideable_type,
    COUNT(*) AS ride_count
FROM 
    combined_tripdata
GROUP BY 
    member_casual, rideable_type
ORDER BY 
    member_casual, ride_count DESC;
"""

result = pd.read_sql(query, conn)

# Find the most used bike type for each group (member and casual)
most_used_bike_types = result.groupby('member_casual').first().reset_index()

print(most_used_bike_types)

In [ ]:
query = """
SELECT
member_casual,
COUNT(*) AS classic_bike_count
FROM
    combined_tripdata
WHERE
rideable_type = 'classic_bike'
GROUP BY
member_casual;
"""
result = pd.read_sql(query, conn)

print(result)

### Looking for Trends in different areas 
Looking to see what areas could be converted from casual riders to memberships. Ranking the top 10 stations and seeing if there is an overlap in casual riders and memberships.

In [ ]:
query = """
WITH RankedStations AS (
    SELECT 
        member_casual,
        start_station_name,
        COUNT(*) AS trip_count,
        ROW_NUMBER() OVER (PARTITION BY member_casual ORDER BY COUNT(*) DESC) AS rank
    FROM 
        combined_tripdata
    GROUP BY 
        member_casual, start_station_name
)
SELECT 
    member_casual,
    start_station_name,
    trip_count
FROM 
    RankedStations
WHERE 
    rank <= 10
ORDER BY 
    member_casual, rank;
"""

result = pd.read_sql(query, conn)

print(result)

### Monthly Trends for Members vs Casual 
Now I want to see trend over time for members vs casual riders. I will start with monthly.

In [ ]:
import matplotlib.pyplot as plt

query = """
SELECT 
    strftime('%Y-%m', started_at) AS month,
    member_casual,
    COUNT(*) AS trip_count
FROM 
    combined_tripdata
GROUP BY 
    month, member_casual
ORDER BY 
    month, member_casual;
"""

# Execute the query and fetch the results
monthly_trends = pd.read_sql(query, conn)

# Ensure the month column is a string and handle missing values
monthly_trends['month'] = monthly_trends['month'].astype(str)
monthly_trends['trip_count'] = monthly_trends['trip_count'].fillna(0)

# Plot the monthly trends
plt.figure(figsize=(12, 6))
for member_type in monthly_trends['member_casual'].unique():
    data = monthly_trends[monthly_trends['member_casual'] == member_type]
    plt.plot(data['month'], data['trip_count'], marker='o', label=member_type)

plt.xlabel('Month')
plt.ylabel('Number of Trips')
plt.title('Monthly Trip Trends by Member Type')
plt.xticks(rotation=45)
plt.legend(title='Member Type')
plt.grid(True)
plt.show()



### Daily Trends by member type

In [ ]:
query = """
SELECT 
    strftime('%w', started_at) AS day_of_week,
    member_casual,
    COUNT(*) AS trip_count
FROM 
    combined_tripdata
GROUP BY 
    day_of_week, member_casual
ORDER BY 
    day_of_week, member_casual;
"""

# Execute the query and fetch the results
daily_trends = pd.read_sql(query, conn)

# Map day_of_week to actual day names and handle missing values
day_map = {
    '0': 'Sunday', 
    '1': 'Monday', 
    '2': 'Tuesday', 
    '3': 'Wednesday', 
    '4': 'Thursday', 
    '5': 'Friday', 
    '6': 'Saturday'
}
daily_trends['day_of_week'] = daily_trends['day_of_week'].map(day_map)
daily_trends['day_of_week'] = pd.Categorical(daily_trends['day_of_week'], categories=[
    'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'], ordered=True)
daily_trends['trip_count'] = daily_trends['trip_count'].fillna(0)

# Ensure 'day_of_week' column contains no missing values
daily_trends = daily_trends.dropna(subset=['day_of_week'])

# Plot the daily trends
plt.figure(figsize=(12, 6))
for member_type in daily_trends['member_casual'].unique():
    data = daily_trends[daily_trends['member_casual'] == member_type]
    plt.plot(data['day_of_week'], data['trip_count'], marker='o', label=member_type)

plt.xlabel('Day of the Week')
plt.ylabel('Number of Trips')
plt.title('Daily Trip Trends by Member Type')
plt.legend(title='Member Type')
plt.grid(True)
plt.show()


### Looking at the top 10 stations for number of rides
Then showing how many of those rides are by casual riders or members
Looking for target areas to convert casual riders to memberships

In [ ]:
import seaborn as sns

# SQL query to get the top 10 stations and their rider counts by member type
query = """
WITH TopStations AS (
    SELECT 
        start_station_name,
        COUNT(*) AS total_rides
    FROM 
        combined_tripdata
    GROUP BY 
        start_station_name
    ORDER BY 
        total_rides DESC
    LIMIT 10
)
SELECT 
    ts.start_station_name,
    ts.total_rides,
    ct.member_casual,
    COUNT(ct.member_casual) AS rider_count
FROM 
    TopStations ts
JOIN 
    combined_tripdata ct
ON 
    ts.start_station_name = ct.start_station_name
GROUP BY 
    ts.start_station_name, ct.member_casual, ts.total_rides
ORDER BY 
    ts.total_rides DESC, ts.start_station_name;
"""

# Execute the query and fetch the results
top_stations = pd.read_sql(query, conn)

# Pivot the results for better visualization
pivot_table = top_stations.pivot(index='start_station_name', columns='member_casual', values='rider_count').fillna(0)
pivot_table['total_rides'] = pivot_table.sum(axis=1)
pivot_table = pivot_table.sort_values('total_rides', ascending=False)

# Plot the results
pivot_table[['casual', 'member']].plot(kind='bar', stacked=True, figsize=(12, 6))
plt.xlabel('Start Station Name')
plt.ylabel('Number of Riders')
plt.title('Top 10 Stations by Number of Riders (Casual vs Member)')
plt.legend(title='Rider Type')
plt.xticks(rotation=45, ha='right')
plt.grid(True)
plt.tight_layout()
plt.show()

### Summary

The season for the bike share satrts in March and peaks around July and August with members lasting longer into September. 
Casual riders tend to use the bike share on the weekends with members using the bike share during the week. There are large areas for conversion at the #1 ranked station with 4000 casual rides taken from that station and 6000 membership rides. It looks like this area could be a good target area for commuters. Same with the #2 ranked station. 
There are twice as many membership riders as there are casual riders, showing a trend moving from casual riders to memberships. 
Both members and casual riders use electric bikes over classic bikes. With casual riders taking rides twice as long as members. 

### Recommendations 

Target Top 4 stations to convert casual members to memberships. Give discounted rate to commuters in that area for membership. Adverize in local businesses around top stations. Advertize in events around those areas. 